In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd
import datetime
import csv
import os
import psycopg2
import numpy as np
import schedule
from psycopg2.extras import DateTimeRange, DateTimeTZRange

print("import finish")


import finish


In [2]:


def run_script():
    driver = webdriver.Chrome(executable_path=r"C:\Chrome\chromedriver.exe")

    url = 'https://www.linkedin.com/login'
    driver.get(url)
    time.sleep(2)

    #import username and password
    credential = open('.env')
    line = credential.readlines()
    username = line[0]
    password = line[1]
    #key in username
    email_field = driver.find_element(By.ID, 'username')
    email_field.send_keys(username)
    time.sleep(3)
    #key in password
    password_field = driver.find_element(By.NAME, 'session_password')
    password_field.send_keys(password)
    time.sleep(2)
    #sign in 
    login_field = driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button')
    login_field.click()
    time.sleep(3)

    url = "https://www.linkedin.com/feed/hashtag/recrutement/"
    driver.get(url)
    time.sleep(2)


    start = time.time()
    initialScroll = 0
    finalScroll = 10000
    scroll_time = 150 # in seconds

    while (time.time() - start) < scroll_time:
        driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
        initialScroll = finalScroll
        finalScroll += 10000
        time.sleep(3)
        
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div[3]/div/div[2]/div/button')))
            show_more_button.click()
            time.sleep(5)
        except:
            pass
        

    infoname = []
    job_actor = []
    mails = []
    date_post = []
    description = []
    hashtag = []
    date_now = [] 
    ########
    page_source = bs(driver.page_source,"html.parser")
    all_post = page_source.find('div', class_ ="scaffold-finite-scroll__content")
    postts = all_post.find_all('div',class_ = "feed-shared-update-v2 feed-shared-update-v2--minimal-padding full-height relative artdeco-card")

    for post in postts:
            mailTo = post.find_all('a')
            tmp_hashtag = []
            for a in mailTo:

                if str(a.get('href')).startswith("mailto:") and a.get('href') not in mails: 
                    mails.append(a.get_text().strip())
                    name_info = post.find('span',class_ ="update-components-actor__name t-14 t-bold hoverable-link-text t-black")
                    infoname.append(name_info.find('span').get_text().strip())    
                    job_actor.append(post.find('span',class_="update-components-actor__description t-black--light t-12 t-normal").get_text().strip())
                    description.append(post.find('span',class_="break-words").get_text().strip())
                    
                    date = post.find('div', class_="update-components-text-view break-words")
                    if date is not None:
                        date_span = date.find('span', class_="visually-hidden")
                        if date_span is not None:
                            date_post.append(date_span.get_text().strip())
                        else:
                            pass
                    else:
                        pass
                    
                    ###############
                    for b in mailTo:
                        if str(b.get('href')).startswith("https://www.linkedin.com/feed/hashtag/") and b.get('href') not in tmp_hashtag:
                                tmp_hashtag.append(b.get_text().strip())
                        else:
                            pass
                    hashtag.append(tmp_hashtag) 
                    
                    ###############
                    
                else : 
                    pass
                
    
    #####################################

    prefixes = ['1 minute ago', 'Just', '1 hour ago']
    date_ago = []
    for k in date_post:
        if k.startswith(tuple(prefixes)):
            if '1 hour' in k:
                date_ago.append('1 hours ago')
            else:
                date_ago.append('1 minutes ago')
        else:
            date_ago.append(k) 
            
    print(date_ago) 
    #### time to timestamp 

    date_now = []
    for datee in date_ago:
        # Check if input data matches expected pattern
        if re.match(r'\d+[mhdw] •', datee):
            # Split input data by a space
            datee_split = datee.split(' ')[0].strip().split()
            time_duration = None
            try:
                time_duration = int(datee_split[0])
            except ValueError:
                
                pass
            if time_duration is not None:
                dt = datetime.timedelta(minutes=time_duration)
            else:
                time_unit = datee_split[0][-1]
                if time_unit == 'm':
                    time_duration = int(datee_split[0][:-1])
                    dt = datetime.timedelta(minutes=time_duration)
                elif time_unit == 'h':
                    time_duration = int(datee_split[0][:-1])
                    dt = datetime.timedelta(hours=time_duration)
                elif time_unit == 'd':
                    time_duration = int(datee_split[0][:-1])
                    dt = datetime.timedelta(days=time_duration)
                elif time_unit == 'w':  # add support for weeks
                    time_duration = int(datee_split[0][:-1])
                    dt = datetime.timedelta(weeks=time_duration)
                else:
                    raise ValueError(f"Invalid time unit '{time_unit}'. Expected 'm', 'h', 'd', or 'w'.")
            past_time = datetime.datetime.now() - dt
            date_now.append(past_time.strftime('%Y-%m-%d %H:%M:%S'))
        else:
            print(f"Invalid input data: {datee}")

    print(date_now)

    print(mails)
    #####################################
    unwanted_domains = {'gmail', 'yahoo', 'live', 'hotmail', 'outlook'}
    domains = []
    for email in mails:
        domain = email.split('@')[1].split('.')[0]
        if domain not in unwanted_domains:
            domains.append(domain)
        else:
            domains.append(' ')
    name_societe = domains

    print(name_societe)  
    #####################################                      
    talent = []
    #hedhy nhotouha eme menech bech nestaamlouha khater hachetna ken bel loop ahna 
    for _ in range(len(hashtag)):
        talent.append([])

    talent_hashtags = []
    with open('words_job_talent.txt', 'r') as f:
        for line in f:
            for tag in line.strip().split(','):
                talent_hashtags.append(tag.strip('#'))

    for i, hashtags in enumerate(hashtag):
        for tag in hashtags:
            if tag.strip('#') in talent_hashtags:
                talent[i].append(tag.strip('#'))

    for i in range(len(talent)):
        talent_hashtag_str = ','.join(talent[i])
        talent[i] = talent_hashtag_str
    #####################################
    numero = []
    with open('job_descriptions.txt', 'a',encoding="utf-8") as f:
        for line in description:
            
            f.write('\n')
            f.write(line)
            f.write('\n')
            phone_regex = r"\b(\d{2}\s\d{2}\s\d{2}\s\d{2}\s\d{2}|\d{3}-\d{3}-\d{4}|\d{2}\.\d{2}\.\d{2}\.\d{2}\.\d{2}|\d{2} \d{2} \d{2} \d{2} \d{2})\b"
            matches = re.findall(phone_regex, line)
            if len(matches) > 0:
                numero.append(matches[0])
            else:
                numero.append("")
            
    #####################################    
        
    df = pd.DataFrame(list(zip(infoname,job_actor,mails,date_now,numero,name_societe,talent,hashtag,description)),
        columns =['Nom Actor :', 'His Job :','mail','date','num','company','speciality','Hachtags :','description'])
    df.to_csv('scraping.csv',mode='a', index=False, encoding="utf-8",sep=',')




    ## raja3ha utf-32 juste kaed njarab b utf-8 w badel w bel a 


    df = pd.read_csv('scraping.csv', encoding="utf-8", sep=',', header=0)
    df.drop_duplicates(subset=['Nom Actor :', 'mail'], keep='first', inplace=True)
    df = df.iloc[:-1]  # remove last row

    # Check if 'Date Post :' column exists
    if 'date' in df.columns:
        try:
            df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
        except ValueError:
            # Handle the case when the format doesn't match
            df['date'] = pd.to_datetime(df['date'], errors='coerce')

    df.to_csv('scraping_updated1.csv', mode='w', index=False, encoding="utf-8", sep=',')


    input_file = 'scraping_updated1.csv'
    category_file = 'category.csv'
    job_file = 'job.csv'
    company_file = 'company.csv'

    # Define the columns to extract from the input file
    category_columns = ['speciality']
    job_columns = [ 'mail','date','num','company','speciality','description']
    company_columns = ['company', 'num']

    # Read the input file and extract the desired columns
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        
        category_data = []
        job_data = []
        company_data = []
        
        for row in reader:
            category_data.append([row[column] for column in category_columns])
            job_data.append([row[column] for column in job_columns])
            company_data.append([row[column] for column in company_columns])

    # Write the extracted data to the output files
    with open(category_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(category_columns)
        writer.writerows(category_data)

    with open(job_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(job_columns)
        writer.writerows(job_data)

    with open(company_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(company_columns)
        writer.writerows(company_data)

    print("Data has been extracted and written to the output files.")


    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="MyKelMission",
        user="postgres",
        password="97955187"
    )

    # Create a cursor object
    cursor = conn.cursor()

    # Create category table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS category (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255) NOT NULL
        )
    """)

    # Import category.csv
    category_data = pd.read_csv('category.csv')
    category_data = category_data[['speciality']]
    category_data = category_data.dropna()  # Remove rows with missing values

    # Insert category data into the database
    for index, row in category_data.iterrows():
        cursor.execute("INSERT INTO category (name) VALUES (%s)", (row['speciality'],))

    # Create company table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS company (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            numero VARCHAR(255) NOT NULL,
            mail VARCHAR(255)
        )
    """)

    # Import company.csv
    company_data = pd.read_csv('company.csv')
    company_data = company_data[['company', 'num']]

    # Insert company data into the database
    for index, row in company_data.iterrows():
        cursor.execute("INSERT INTO company (name, numero) VALUES (%s, %s)", (row['company'], row['num']))

    # Create job table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS job (
            id SERIAL PRIMARY KEY,
            mail VARCHAR(255),
            date DATE,
            num VARCHAR(255),
            company_id INTEGER,
            speciality VARCHAR(255),
            FOREIGN KEY (company_id) REFERENCES company (company_id),
            FOREIGN KEY (category_id) REFERENCES category (id),
            description VARCHAR(255)
        )
    """)

    # Import job.csv
    job_data = pd.read_csv('job.csv')
    job_data = job_data[['mail', 'date', 'num', 'company', 'speciality', 'description']]

    # Convert 'date' column to pandas DateTime type and handle missing values
    job_data['date'] = pd.to_datetime(job_data['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    # Insert job data into the database
    for index, row in job_data.iterrows():
        # Get the company_id based on the company name
        cursor.execute("SELECT company_id FROM company WHERE name = %s", (row['company'],))
        company_id = cursor.fetchone()[0]
        
        # Check if speciality is NaN and handle it appropriately
        if pd.isnull(row['speciality']):
            category_id = None
        else:
            # Get the category_id based on the category name
            cursor.execute("SELECT category_id FROM category WHERE name = %s", (row['speciality'],))
            category_id = cursor.fetchone()[0]
        
        # Check if date is NaT (Not a Time) and handle it appropriately
        if pd.isnull(row['date']):
            date_value = None
        else:
            date_value = row['date']

        cursor.execute("INSERT INTO job (mail, date, num, company_id, category_id, speciality, description) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                    (row['mail'], date_value, row['num'], company_id, category_id, row['speciality'], row['description']))

    # Get the count of emails obtained
    mail_count = len(mails)

    # Get the current date
    current_date = pd.to_datetime('today').date()

    # Insert the mail count and current date into the log table
    cursor.execute("INSERT INTO log (number_jobs, date_log) VALUES (%s, %s)", (mail_count, current_date))
     
    
    # Commit the changes to the database
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    
print("Script executed")

schedule.every().day.at("15:40").do(run_script)

while True:
    schedule.run_pending()
    time.sleep(1)

Script executed


C:\Users\Asus\AppData\Local\Temp\ipykernel_16204\3070219728.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Chrome\chromedriver.exe")
Incompatible release of chromedriver (version 109.0.5414.74) detected in PATH: C:\webdrivers\chromedriver.exe


['1d •', '37m •', '14m •', '35m •', '25m •', '30m •', '39m •', '3w •', '5h •', '1h • Edited •', '44m •', '23h •', '2h • Edited •', '9m •', '9m •', '21m •', '14m •', '12m •', '12m •', '8m •']
['2023-07-24 15:44:13', '2023-07-25 15:07:13', '2023-07-25 15:30:13', '2023-07-25 15:09:13', '2023-07-25 15:19:13', '2023-07-25 15:14:13', '2023-07-25 15:05:13', '2023-07-04 15:44:13', '2023-07-25 10:44:13', '2023-07-25 14:44:13', '2023-07-25 15:00:13', '2023-07-24 16:44:13', '2023-07-25 13:44:13', '2023-07-25 15:35:13', '2023-07-25 15:35:13', '2023-07-25 15:23:13', '2023-07-25 15:30:13', '2023-07-25 15:32:13', '2023-07-25 15:32:13', '2023-07-25 15:36:13']
['i.koudjonou@montpellier-bs.com', 'recrutement@alsacerhinbrisach.fr', 'recrutement@next-showroom.com', 'recrutement@gem-manutention.com', 'achelidz@yahoo.fr', 'francois.danel@earthwake-entreprise.fr', 'marketing@izyworkci.com', 'agence.cherbourg@lassarat.com', 'matteo@ift-filters.com', 'elodie.allasia@gmail.com', 'pierre.vandeputte@icloud.com', 